# Import the Zenbase Library


In [13]:
import sys
import subprocess

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}: {e}")
        raise

def install_packages(packages):
    for package in packages:
        install_package(package)

try:
    # Check if running in Google Colab
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install the zenbase package if running in Google Colab
    # install_package('zenbase')
    # Install the zenbse package from a GitHub branch if running in Google Colab
    install_package('git+https://github.com/zenbase-ai/lib.git@main#egg=zenbase&subdirectory=py')

    # List of other packages to install in Google Colab
    additional_packages = [
        'python-dotenv',
        'lunary',
        'openai',
        'langchain',
        'langchain_openai'
    ]
    
    # Install additional packages
    install_packages(additional_packages)

# Now import the zenbase library
try:
    import zenbase
except ImportError as e:
    print("Failed to import zenbase: ", e)
    raise

# Configure the Environment

In [1]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["LUNARY_PUBLIC_KEY"] = "..."

load_dotenv(Path("../../.env.test"), override=True)

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

# Initial Setup

In [3]:
from openai import OpenAI
import lunary

openai = OpenAI()
lunary.monitor(openai)

# What you already have should look like below:

## Your OpenAI Call should look like this with LangChain (It could be with OpenAI too, doesn't matter)

In [4]:
def langchain_chain(inputs):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        ),
        ("user", "{question}"),
    ]

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(inputs)
    return answer.split("#### ")[-1]

## Your evaluation should look like this:

Note: 'exact-match' is the evaluation metric that you have defined in the lunry by yourself.

In [5]:
evalset = lunary.get_dataset("gsm8k-evalset")

scores = []
for item in evalset:
    answer = langchain_chain(item.input)
    passed, results = lunary.evaluate(
        "exact-match",
        input=item.input,
        output=answer,
        ideal_output=item.ideal_output.split("#### ")[-1],
    )
    scores.append(int(passed))

print("Average score", sum(scores) / len(scores))

Average score 0.2


# How you should do the few-shot learning

## Rewrite your langchain_chain function to use the `zenbase` decorators

In [6]:
from zenbase.types import LMRequest, deflm


@deflm
def zen_chain(request: LMRequest):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        )
    ]

    for demo in request.zenbase.task_demos:
        messages += [
            ("user", demo.inputs),
            ("assistant", demo.outputs),
        ]

    messages.append(("user", "{question}"))

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(request.inputs)
    return answer.split("#### ")[-1]

## Optimize the few-shot learning

### Define your optimizer:

In [10]:
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.adaptors.lunary import ZenLunary

demoset = ZenLunary.dataset_to_demos(lunary.get_dataset("gsmk8k-train-set"))
optimizer = LabeledFewShot(demoset=demoset, shots=3)



### Perform the optimization

In [ ]:
best_fn, candidate_results, _ = optimizer.perform(
    zen_chain,
    evaluator=ZenLunary.metric_evaluator(
        checklist="exact-match",
        evalset=evalset,
        concurrency=5,
    ),
    samples=4,
    rounds=1,
)

### Use the best function

In [11]:
output = best_fn({"question": "What is 2+2?"})
output

'4'

### Save the best function

In [12]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
zen_chain.zenbase = pickle.loads(pickled_zenbase)

zen_chain({"question": "What is 2 + 2?"}) # uses the best few-shot demos

'4'